# Re-Align 2026: Paper Reproduction

**Title**: Spectral Steering as a Bridge for Functional Fractures
**Experiment**: Rigorous Statistical Validation (N=300)

This notebook reproduces **Table 1** and **Figure 2** (Phase Space Analysis) from the paper submission.

### Summary of Results
We compare **Spectral Steering (Layer 24)** against a **Baseline** and two strict controls:
1.  **Control A (Random)**: Steering with a random vector of matched norm. (Tests Specificity - prevents "magnitude hacking").
2.  **Control B (Anti)**: Steering to *minimize* connectivity. (Tests Causality).

**Key Finding**: Spectral Steering improves Logic Accuracy from **51.7%** to **78.3%** ($p < 10^{-7}$), while Random Steering destroys performance (**11%**).

In [ ]:
import json
import numpy as np
import math
import matplotlib.pyplot as plt
from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.proportion import proportion_confint

RESULTS_FILE = "../output/rigorous_results_N300.json"
DATA_FILE = "../data/logic_dataset.json"
PLOT_FILE = "../analysis/phase_space_plot.png"
CONDITIONS = ["Baseline", "Control_Random", "Control_Anti", "Spectral"]

print("Environment Ready.")

## 1. Load Data
Loading the N=300 dataset and the experimental results.

In [ ]:
try:
    with open(RESULTS_FILE, 'r') as f:
        data = json.load(f)
    print(f"Loaded results for N={data['N']} samples.")
except FileNotFoundError:
    print("Results file not found! Please run 'python experiments/15_rigorous_validation.py' first.")
    # Optional: Logic to run the experiment if missing could go here.

## 2. Statistical Analysis (Reproducing Table 1)
We calculate:
- **Accuracy**
- **95% Confidence Interval** (Wilson Score)
- **Effect Size** (Cohen's h)
- **Significance** (McNemar's Test for paired data)

In [ ]:
def calculate_cohens_h(p1, p2):
    return 2 * (math.asin(math.sqrt(p2)) - math.asin(math.sqrt(p1)))

print(f"{'Condition':<20} | {'Acc':<8} | {'95% CI (Wilson)':<18} | {'Cohen\'s h':<10} | {'McNemar p':<12}")
print("-" * 85)

baseline_results = data["Baseline"]["results"]
baseline_acc = data["Baseline"]["accuracy"]
N = data["N"]

for cond in CONDITIONS:
    cond_data = data.get(cond)
    if not cond_data: continue
    
    results = cond_data["results"]
    acc = cond_data["accuracy"]
    correct_count = int(acc * N)
    
    # CI
    ci_low, ci_high = proportion_confint(correct_count, N, method='wilson')
    ci_str = f"[{ci_low:.1%}, {ci_high:.1%}]"
    
    # Stats
    if cond != "Baseline":
        h_val = calculate_cohens_h(baseline_acc, acc)
        
        # McNemar Matrix
        both_fail = 0
        base_fail_cond_pass = 0 # The Wins
        base_pass_cond_fail = 0 # The Regressions
        both_pass = 0
        
        for pid, base_res in baseline_results.items():
            cond_res = results.get(pid, False)
            if base_res and cond_res: both_pass += 1
            elif not base_res and not cond_res: both_fail += 1
            elif not base_res and cond_res: base_fail_cond_pass += 1
            elif base_res and not cond_res: base_pass_cond_fail += 1
        
        table = [[both_fail, base_fail_cond_pass],
                 [base_pass_cond_fail, both_pass]]
                 
        if base_fail_cond_pass + base_pass_cond_fail > 0:
            mcnemar_res = mcnemar(table, exact=True)
            p_val = mcnemar_res.pvalue
        else:
            p_val = 1.0
            
        p_str = f"{p_val:.2e}" if p_val < 0.001 else f"{p_val:.2f}"
        h_str = f"{h_val:.2f}"
    else:
        p_str = "——"
        h_str = "——"

    print(f"{cond:<20} | {acc:.1%}    | {ci_str:<18} | {h_str:<10} | {p_str:<12}")

## 3. Visualization: The Phase Space (Figure 2)
Loading the generated Phase Space plot.

In [ ]:
from IPython.display import Image, display
display(Image(filename=PLOT_FILE))